## Load the data

In [13]:
from dagster import AssetKey
from pudl.etl import defs
import pandas as pd

raw_df = defs.load_asset_value(AssetKey("raw_phmsagas__yearly_distribution"))
print(f"Earliest reported year available in asset: {raw_df.report_year.min()}")
print(f"Latest reported year available in asset: {raw_df.report_year.max()}")
raw_df.report_year.unique()


No dagster instance configuration file (dagster.yaml) found at /Users/sam/Documents/pudl-data/dagster_home. Defaulting to loading and storing all metadata with /Users/sam/Documents/pudl-data/dagster_home. If this is the desired behavior, create an empty dagster.yaml file in /Users/sam/Documents/pudl-data/dagster_home.
2024-09-30 17:09:20 -0400 - dagster - DEBUG - system - Loading file from: /Users/sam/Documents/pudl-data/dagster_home/storage/raw_phmsagas__yearly_distribution using PickledObjectFilesystemIOManager...


Earliest reported year available in asset: 94.0
Latest reported year available in asset: 2023.0


array([1990., 1991., 1992., 1993.,   94.,   95.,   96.,   97., 1998.,
       1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007.,
       2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016.,
       2017., 2018., 2019., 2020., 2021., 2022., 2023.])

## Data exploration

In [2]:
raw_df.head()

,additional_information,all_known_leaks_scheduled_for_repair,all_leaks_construction_defect_mains,all_leaks_construction_defect_services,all_leaks_corrosion_failure_mains,all_leaks_corrosion_failure_services,all_leaks_equipment_failure_mains,all_leaks_equipment_failure_services,all_leaks_excavation_damage_mains,all_leaks_excavation_damage_services,all_leaks_incorrect_operation_mains,all_leaks_incorrect_operation_services,all_leaks_material_defect_mains,all_leaks_material_defect_services,all_leaks_natural_force_damage_mains,all_leaks_natural_force_damage_services,all_leaks_other_mains,all_leaks_other_outside_force_mains,all_leaks_other_outside_force_services,all_leaks_other_services,all_leaks_outside_force_mains,all_leaks_outside_force_services,all_leaks_pipe_weld_joint_failure_mains,all_leaks_pipe_weld_joint_failure_services,all_leaks_third_party_mains,all_leaks_third_party_services,average_service_length_feet,commodity,correction_date,data_date,data_maturity,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,federal_land_leaks_repaired_or_scheduled,filing_date,form_revision,hazardous_leaks_corrosion_failure_mains,hazardous_leaks_corrosion_failure_services,hazardous_leaks_equipment_failure_mains,hazardous_leaks_equipment_failure_services,hazardous_leaks_excavation_damage_mains,hazardous_leaks_excavation_damage_services,hazardous_leaks_incorrect_operation_mains,hazardous_leaks_incorrect_operation_services,hazardous_leaks_mechanical_joint_failure,hazardous_leaks_natural_force_damage_mains,hazardous_leaks_natural_force_damage_services,hazardous_leaks_other_mains,hazardous_leaks_other_outside_force_mains,hazardous_leaks_other_outside_force_services,hazardous_leaks_other_services,hazardous_leaks_pipe_weld_joint_failure_mains,hazardous_leaks_pipe_weld_joint_failure_services,headquarters_address_city,headquarters_address_county,headquarters_address_state,headquarters_address_street,headquarters_address_zip,log,main_1940s_miles,main_1950s_miles,main_1960s_miles,main_1970s_miles,main_1980s_miles,main_1990s_miles,main_2000s_miles,main_2010s_miles,main_2020s_miles,main_abs_2_in_or_less_miles,main_abs_2_to_4_in_miles,main_abs_4_to_8_in_miles,main_abs_8_to_12_in_miles,main_abs_over_12_in_miles,main_abs_total_miles,main_abs_unknown_miles,main_all_materials_2_in_or_less_miles,main_all_materials_2_to_4_in_miles,main_all_materials_4_to_8_in_miles,main_all_materials_8_to_12_in_miles,main_all_materials_over_12_in_miles,main_all_materials_total_miles,main_all_materials_unknown_miles,main_all_time_miles,main_cast_or_wrought_iron_2_in_or_less_miles,main_cast_or_wrought_iron_2_to_4_in_miles,main_cast_or_wrought_iron_4_to_8_in_miles,main_cast_or_wrought_iron_8_to_12_in_miles,main_cast_or_wrought_iron_miles,main_cast_or_wrought_iron_over_12_in_miles,main_cast_or_wrought_iron_total_miles,main_cast_or_wrought_iron_unknown_miles,main_cathodically_protected_steel_bare_miles,main_cathodically_protected_steel_coated_miles,main_copper_2_in_or_less_miles,main_copper_2_to_4_in_miles,main_copper_4_to_8_in_miles,main_copper_8_to_12_in_miles,main_copper_miles,main_copper_over_12_in_miles,main_copper_total_miles,main_copper_unknown_miles,main_ductile_iron_2_in_or_less_miles,main_ductile_iron_2_to_4_in_miles,main_ductile_iron_4_to_8_in_miles,main_ductile_iron_8_to_12_in_miles,main_ductile_iron_miles,main_ductile_iron_over_12_in_miles,main_ductile_iron_total_miles,main_ductile_iron_unknown_miles,main_other_2_in_or_less_miles,main_other_2_to_4_in_miles,main_other_4_to_8_in_miles,main_other_8_to_12_in_miles,main_other_alt_2_in_or_less_miles,main_other_alt_2_to_4_in_miles,main_other_alt_4_to_8_in_miles,main_other_alt_8_to_12_in_miles,main_other_alt_miles,main_other_alt_over_12_in_miles,main_other_alt_total_miles,main_other_alt_unknown_miles,main_other_material_detail_miles,main_other_miles,main_other_over_12_in_miles,main_other_plastic_2_in

In [3]:
def sorted_value_printer(column):
    # Assuming 'raw_df' is your DataFrame
    sorted_values = sorted(raw_df[column])

    # Pretty print the sorted columns
    for val in sorted_values:
        print(val)

In [4]:
print(raw_df.filing_date.max())
print(raw_df.filing_date.min())

print(raw_df.report_date.max())
print(raw_df.report_date.min())

print(raw_df.report_year.max())
print(raw_df.report_year.min())

2024-07-31 17:28:42
1991-01-25 00:00:00
2024-07-31 14:16:33
2000-01-05 00:00:00
2023.0
94.0


In [5]:
raw_df[raw_df.operator_id_phmsa==383][['operator_id_phmsa','operator_name_phmsa']]

,operator_id_phmsa,operator_name_phmsa
25,383.0,AMARILLO NATURAL GAS INC
1539,383.0,AMARILLO NATURAL GAS INC
3627,383.0,AMARILLO NATURAL GAS INC
4645,383.0,AMARILLO NATURAL GAS INC
6214,383.0,AMARILLO NATURAL GAS INC ...
...,...,...
46339,383.0,AMARILLO NATURAL GAS INC
47782,383.0,AMARILLO NATURAL GAS INC
47783,383.0,AMARILLO NATURAL GAS INC
49229,383.0,AMARILLO NATURAL GAS INC


In [6]:
raw_df[raw_df.operator_id_phmsa==383.0].head()

,additional_information,all_known_leaks_scheduled_for_repair,all_leaks_construction_defect_mains,all_leaks_construction_defect_services,all_leaks_corrosion_failure_mains,all_leaks_corrosion_failure_services,all_leaks_equipment_failure_mains,all_leaks_equipment_failure_services,all_leaks_excavation_damage_mains,all_leaks_excavation_damage_services,all_leaks_incorrect_operation_mains,all_leaks_incorrect_operation_services,all_leaks_material_defect_mains,all_leaks_material_defect_services,all_leaks_natural_force_damage_mains,all_leaks_natural_force_damage_services,all_leaks_other_mains,all_leaks_other_outside_force_mains,all_leaks_other_outside_force_services,all_leaks_other_services,all_leaks_outside_force_mains,all_leaks_outside_force_services,all_leaks_pipe_weld_joint_failure_mains,all_leaks_pipe_weld_joint_failure_services,all_leaks_third_party_mains,all_leaks_third_party_services,average_service_length_feet,commodity,correction_date,data_date,data_maturity,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,federal_land_leaks_repaired_or_scheduled,filing_date,form_revision,hazardous_leaks_corrosion_failure_mains,hazardous_leaks_corrosion_failure_services,hazardous_leaks_equipment_failure_mains,hazardous_leaks_equipment_failure_services,hazardous_leaks_excavation_damage_mains,hazardous_leaks_excavation_damage_services,hazardous_leaks_incorrect_operation_mains,hazardous_leaks_incorrect_operation_services,hazardous_leaks_mechanical_joint_failure,hazardous_leaks_natural_force_damage_mains,hazardous_leaks_natural_force_damage_services,hazardous_leaks_other_mains,hazardous_leaks_other_outside_force_mains,hazardous_leaks_other_outside_force_services,hazardous_leaks_other_services,hazardous_leaks_pipe_weld_joint_failure_mains,hazardous_leaks_pipe_weld_joint_failure_services,headquarters_address_city,headquarters_address_county,headquarters_address_state,headquarters_address_street,headquarters_address_zip,log,main_1940s_miles,main_1950s_miles,main_1960s_miles,main_1970s_miles,main_1980s_miles,main_1990s_miles,main_2000s_miles,main_2010s_miles,main_2020s_miles,main_abs_2_in_or_less_miles,main_abs_2_to_4_in_miles,main_abs_4_to_8_in_miles,main_abs_8_to_12_in_miles,main_abs_over_12_in_miles,main_abs_total_miles,main_abs_unknown_miles,main_all_materials_2_in_or_less_miles,main_all_materials_2_to_4_in_miles,main_all_materials_4_to_8_in_miles,main_all_materials_8_to_12_in_miles,main_all_materials_over_12_in_miles,main_all_materials_total_miles,main_all_materials_unknown_miles,main_all_time_miles,main_cast_or_wrought_iron_2_in_or_less_miles,main_cast_or_wrought_iron_2_to_4_in_miles,main_cast_or_wrought_iron_4_to_8_in_miles,main_cast_or_wrought_iron_8_to_12_in_miles,main_cast_or_wrought_iron_miles,main_cast_or_wrought_iron_over_12_in_miles,main_cast_or_wrought_iron_total_miles,main_cast_or_wrought_iron_unknown_miles,main_cathodically_protected_steel_bare_miles,main_cathodically_protected_steel_coated_miles,main_copper_2_in_or_less_miles,main_copper_2_to_4_in_miles,main_copper_4_to_8_in_miles,main_copper_8_to_12_in_miles,main_copper_miles,main_copper_over_12_in_miles,main_copper_total_miles,main_copper_unknown_miles,main_ductile_iron_2_in_or_less_miles,main_ductile_iron_2_to_4_in_miles,main_ductile_iron_4_to_8_in_miles,main_ductile_iron_8_to_12_in_miles,main_ductile_iron_miles,main_ductile_iron_over_12_in_miles,main_ductile_iron_total_miles,main_ductile_iron_unknown_miles,main_other_2_in_or_less_miles,main_other_2_to_4_in_miles,main_other_4_to_8_in_miles,main_other_8_to_12_in_miles,main_other_alt_2_in_or_less_miles,main_other_alt_2_to_4_in_miles,main_other_alt_4_to_8_in_miles,main_other_alt_8_to_12_in_miles,main_other_alt_miles,main_other_alt_over_12_in_miles,main_other_alt_total_miles,main_other_alt_unknown_miles,main_other_material_detail_miles,main_other_miles,main_other_over_12_in_miles,main_other_plastic_2_in

In [7]:
# What values are different when we look at AMARILLO NATURAL GAS INC (ID 383)
filtered_df = raw_df[raw_df.operator_id_phmsa==383.0]

columns_with_differences = filtered_df.loc[:, filtered_df.nunique() > 1]

result_dict = {
    col: ", ".join(map(str, columns_with_differences[col].values))
    for col in columns_with_differences.columns
}

# Step 4: Pretty print the dictionary
import pprint
pprint.pprint(result_dict)
sorted_columns = sorted(raw_df.columns)

# Pretty print the sorted columns
for col in sorted_columns:
    print(col)

{'all_known_leaks_scheduled_for_repair': '0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                                         '0.0, 0.0, 0.0, 0.0',
 'all_leaks_excavation_damage_mains': 'nan, nan, nan, nan, nan, nan, nan, nan, '
                                      'nan, nan, nan, nan, nan, nan, nan, nan, '
                                      'nan, nan, nan, 0.0, 0.0,

In [8]:
# Let's look at all the columns that are available
sorted_cols = sorted(raw_df.columns)

for col in sorted_cols:
    print(col)

additional_information
all_known_leaks_scheduled_for_repair
all_leaks_construction_defect_mains
all_leaks_construction_defect_services
all_leaks_corrosion_failure_mains
all_leaks_corrosion_failure_services
all_leaks_equipment_failure_mains
all_leaks_equipment_failure_services
all_leaks_excavation_damage_mains
all_leaks_excavation_damage_services
all_leaks_incorrect_operation_mains
all_leaks_incorrect_operation_services
all_leaks_material_defect_mains
all_leaks_material_defect_services
all_leaks_natural_force_damage_mains
all_leaks_natural_force_damage_services
all_leaks_other_mains
all_leaks_other_outside_force_mains
all_leaks_other_outside_force_services
all_leaks_other_services
all_leaks_outside_force_mains
all_leaks_outside_force_services
all_leaks_pipe_weld_joint_failure_mains
all_leaks_pipe_weld_joint_failure_services
all_leaks_third_party_mains
all_leaks_third_party_services
average_service_length_feet
commodity
correction_date
data_date
data_maturity
excavation_damage_excavation

## core_phmsagas__yearly_distribution_operators

In [9]:
core_phmsagas__yearly_distribution_operators = raw_df[[
    "report_date",
    "report_number",
    "report_submission_type",
    "report_year",
    "operator_id_phmsa",
    "operator_name_phmsa",
    "office_address_street",
    "office_address_city",
    "office_address_state",
    "office_address_zip",
    "office_address_county",
    "excavation_damage_excavation_practices",
    "excavation_damage_locating_practices",
    "excavation_damage_one_call_notification",
    "excavation_damage_other",
    "excavation_damage_total",
    "excavation_tickets",
    "services_efv_in_system",
    "services_efv_installed",
    "services_shutoff_valve_in_system",
    "services_shutoff_valve_installed",
    "federal_land_leaks_repaired_or_scheduled",
    "percent_unaccounted_for_gas",
    "additional_information",
    "preparer_email",
    "preparer_fax",
    "preparer_name",
    "preparer_phone",
    "preparer_title",
    # Adding these fields temporarily for transformation cleanup
    "headquarters_address_city",
    "headquarters_address_county",
    "headquarters_address_state",
    "headquarters_address_street",
    "headquarters_address_zip"
]]

core_phmsagas__yearly_distribution_operators.head()

,report_date,report_number,report_submission_type,report_year,operator_id_phmsa,operator_name_phmsa,office_address_street,office_address_city,office_address_state,office_address_zip,office_address_county,excavation_damage_excavation_practices,excavation_damage_locating_practices,excavation_damage_one_call_notification,excavation_damage_other,excavation_damage_total,excavation_tickets,services_efv_in_system,services_efv_installed,services_shutoff_valve_in_system,services_shutoff_valve_installed,federal_land_leaks_repaired_or_scheduled,percent_unaccounted_for_gas,additional_information,preparer_email,preparer_fax,preparer_name,preparer_phone,preparer_title,headquarters_address_city,headquarters_address_county,headquarters_address_state,headquarters_address_street,headquarters_address_zip
0,NaT,19901506.0,NaN,1990.0,18.0,"ABBYVILLE, CITY OF",P O BOX 100,ABBYVILLE,KS,67510.0,RENO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3162865404,NaN,DEBRA EHLING,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,19900095.0,NaN,1990.0,27.0,ABITA SPRINGS NAT GAS & WATER,LEVEL STREET,ABITA SPRINGS,LA,70420.0,ST. TAMMANY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,NaN,5048920711,NaN,BARBARA GIANCONTIERI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,19900947.0,NaN,1990.0,45.0,"ADAIRSVILLE, CITY OF",P.O. BOX 830,ADAIRSVILLE,GA,30103.0,BARTOW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,4044221902,NaN,CHRIS STRIPPELHOFF - CONSULTANT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,19901193.0,NaN,1990.0,49.0,"ADAMSVILLE GAS DEPT, TOWN OF",231 EAST MAIN STREET,ADAMSVILLE,TN,38310.0,MCNAIRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.8,NaN,9016324214,NaN,"E. GEORGE LECKNER, JR. - GAS SYSTEM ANALYST",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,19900948.0,NaN,1990.0,54.0,"ADEL GAS DEPT, CITY OF",CITY HALL - P.O. BOX 658,ADEL,GA,31620.0,COOK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.5,NaN,4044221902,NaN,CHRIS STRIPPELHOFF - CONSULTANT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Operator Table Tasks

### Standardize NAs

Based on the analysis below, I'm not seeing any major issues with nans. 

* No empty strings. Just nans.


In [10]:
nan_cols = []
exception_cols = []

# Define custom missing value markers
custom_missing_values = ['', ' ', 'NA', 'N/A', 'NULL', '-', 'None', 'NaN', '?', '*', '#']

# Standardize NAs and analyze columns
for col in core_phmsagas__yearly_distribution_operators.columns:
    try:
        print(f"Analyzing column: {col}")
        
        # Get the column values
        col_data = core_phmsagas__yearly_distribution_operators[col]
        
        # Check if the column is of string (object) type
        if col_data.dtype == 'object':
            # Count rows where the value is NaN, None, empty string, or custom missing values
            none_count = col_data.isna().sum()  # Count None (NaN)
            empty_string_count = (col_data.str.strip() == '').sum()  # Count empty strings
            custom_missing_count = col_data.isin(custom_missing_values).sum()  # Count custom missing values
            
            total_nan_count = none_count + empty_string_count + custom_missing_count
            
            if total_nan_count > 0:
                nan_cols.append(col)
            
            # Output counts
            print(f"Column '{col}' is a string type.")
            if none_count > 0 or empty_string_count > 0 or custom_missing_count > 0:
                print(f"Rows with None values: {none_count}")
                print(f"Rows with empty strings: {empty_string_count}")
                print(f"Rows with custom missing values: {custom_missing_count}")
            else:
                print("Found nothing worth reporting here")
            
        # Check if the column is numeric (int or float)
        elif pd.api.types.is_numeric_dtype(col_data):
            # Output min, max, out-of-range values, and NA count
            print(f"Column '{col}' is numeric.")

            # Count NA values in the column
            na_count = col_data.isna().sum()
            # Count custom missing values in numeric columns (if applicable)
            custom_missing_numeric_count = col_data.isin([0]).sum()  # Assuming 0 is considered a missing value
            
            if na_count > 0 or custom_missing_numeric_count > 0:
                nan_cols.append(col)
            
            # Handle the non-NA data for further analysis
            col_data_cleaned = col_data.dropna()
            
            if not col_data_cleaned.empty:
                # Calculate min and max
                min_val = col_data_cleaned.min()
                max_val = col_data_cleaned.max()
                
                if min_val < 0 or na_count > 0 or custom_missing_numeric_count > 0:
                    print(f"Min value: {min_val}")
                    print(f"Max value: {max_val}")
                    print(f"Rows with NA values: {na_count}")
                    print(f"Custom missing values (e.g., 0): {custom_missing_numeric_count}")
                else:
                    print("Found nothing worth reporting here")
            else:
                print(f"Column '{col}' is numeric but contains only NA values.")
        
        # If the column is of some other type, simply note the type
        else:
            print(f"Column '{col}' is of type {col_data.dtype}.")
        print("")
    except Exception as e:
        exception_cols.append(col)
        print(f"Caught exception for column {col}: {e}\n")
        continue


Analyzing column: report_date
Column 'report_date' is of type datetime64[ns].

Analyzing column: report_number
Column 'report_number' is numeric.
Found nothing worth reporting here

Analyzing column: report_submission_type
Column 'report_submission_type' is a string type.
Rows with None values: 30059
Rows with empty strings: 0
Rows with custom missing values: 0

Analyzing column: report_year
Column 'report_year' is numeric.
Found nothing worth reporting here

Analyzing column: operator_id_phmsa
Column 'operator_id_phmsa' is numeric.
Found nothing worth reporting here

Analyzing column: operator_name_phmsa
Column 'operator_name_phmsa' is a string type.
Found nothing worth reporting here

Analyzing column: office_address_street
Column 'office_address_street' is a string type.
Rows with None values: 59
Rows with empty strings: 3
Rows with custom missing values: 0

Analyzing column: office_address_city
Column 'office_address_city' is a string type.
Rows with None values: 45
Rows with empty

In [11]:
exception_cols

[]

#### percent_unaccounted_for_gas

The one thing that did stick out somewhat was the "percent_unaccounted_for_gas" column having negative values.

This comes from Part G of the PHMSA form, which defines this value as:

```
Unaccounted for gas as a percent of total consumption for the 12 months ending June 30 of the reporting year.
[(Purchased gas + produced gas) minus (customer use + company use + appropriate adjustments)] divided by (customer use + company use + appropriate adjustments) times 100 equals percent unaccounted for.
```

Based on that description, it sounds like negative values here suggest that a company produced/purchased less gas than was consumed. But also it's hard to tell exactly what these figures mean (especially the "appropriate adjustments" part).

So the question is: are we okay with these negative values? Or do we want to manually clean them up somehow?

In [12]:
core_phmsagas__yearly_distribution_operators[core_phmsagas__yearly_distribution_operators.percent_unaccounted_for_gas<0][["percent_unaccounted_for_gas"]]

TypeError: '<' not supported between instances of 'str' and 'int'

### Standardize usage of office vs HQ address over time.

In [ ]:
core_phmsagas__yearly_distribution_operators.head()

In [ ]:
core_phmsagas__yearly_distribution_operators.columns

In [ ]:
core_phmsagas__yearly_distribution_operators[['report_date', 'report_number', 'report_submission_type', 'report_year', 'operator_id_phmsa', 'operator_name_phmsa', 'office_address_street', 'office_address_city', 'office_address_state', 'office_address_zip', 'office_address_county', 'headquarters_address_city', 'headquarters_address_county', 'headquarters_address_state', 'headquarters_address_street', 'headquarters_address_zip', 'additional_information']].head()

In [ ]:
# Assuming 'core_phmsagas__yearly_distribution_operators' is your dataframe
df = core_phmsagas__yearly_distribution_operators

# Selecting the relevant columns
address_cols = [
    'operator_id_phmsa', 'report_year',
    'office_address_street', 'office_address_city', 'office_address_state', 'office_address_zip',
    'headquarters_address_street', 'headquarters_address_city', 'headquarters_address_state', 'headquarters_address_zip'
]

df_addresses = df[address_cols]

# Create a concatenated address string for easier comparison
df_addresses['office_address'] = df_addresses['office_address_street'] + ', ' + df_addresses['office_address_city'] + ', ' + df_addresses['office_address_state'] + ' ' + df_addresses['office_address_zip']
df_addresses['headquarters_address'] = df_addresses['headquarters_address_street'] + ', ' + df_addresses['headquarters_address_city'] + ', ' + df_addresses['headquarters_address_state'] + ' ' + df_addresses['headquarters_address_zip']

# Group by operator and sort by year
df_addresses = df_addresses.sort_values(by=['operator_id_phmsa', 'report_year'])

# Create flags for overlap or switch
df_addresses['address_overlap'] = df_addresses['office_address'] == df_addresses['headquarters_address']
df_addresses['address_switch'] = df_addresses.groupby('operator_id_phmsa').apply(
    lambda x: (x['office_address'].shift() == x['headquarters_address']) & 
              (x['headquarters_address'].shift() == x['office_address'])
).reset_index(drop=True)

# Display the results where overlap or switch occurred
df_overlap_switch = df_addresses[(df_addresses['address_overlap'] == True) | (df_addresses['address_switch'] == True)]


In [ ]:
df_overlap_switch.head()


In [ ]:
# df_overlap_switch[df_overlap_switch.address_switch==True]
df_overlap_switch[(df_overlap_switch.operator_id_phmsa==383)&(core_phmsagas__yearly_distribution_operators.report_year>=2021)]

In [ ]:
print(core_phmsagas__yearly_distribution_operators.report_year.min())
print(core_phmsagas__yearly_distribution_operators.report_year.max())